In [1]:
from datasets import load_dataset, Dataset, load_from_disk
from interprot_sae import get_latents, load_models
from constants import linear_probe_dataset_path

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("SaProtHub/Dataset-GB1-fitness")["train"]

/data/anaconda3/envs/barsaelona_ishan/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-03 17:48:15.440824: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/data/anaconda3/envs/barsaelona_ishan/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
ds.info
ds = Dataset.from_dict(ds[:100])

In [3]:
LAYER = 33
seq = "TTCCPSIVARSNFNVCRLPGTPEALCATYTGCIIIPGATCPGDYAN"
esm_mod, sae_mod = load_models(LAYER)

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
def add_latent(rows):
    seqs = rows["protein"]
    rows["latents"] = get_latents(seqs, LAYER, esm_mod, sae_mod)
    return rows

new_ds = ds.map(add_latent, batched=True)


Map: 100%|██████████| 100/100 [00:01<00:00, 72.91 examples/s]


In [5]:
new_ds.save_to_disk(linear_probe_dataset_path("gb1", LAYER, "sae")) 

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 100/100 [00:00<00:00, 2436.42 examples/s]


In [6]:
new_ds = load_from_disk("test.hf")
new_ds

Dataset({
    features: ['protein', 'label', 'stage', 'latents'],
    num_rows: 100
})